<a href="https://colab.research.google.com/github/visha1Sagar/Text-To-SQL/blob/main/Qwen2_5_7B_semEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### importing libraries

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

### Model

In [6]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.12.12: Fast Qwen2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# from unsloth import FastLanguageModel
# import torch
# max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "Vishal-Sagar/Qwen2.5_7B-text-to-sql-basic",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 24, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 8,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [11]:
from datasets import load_dataset
import json
from datasets import Dataset

alpaca_prompt = """

### Instruction:
You are an SQL Expert. Your task is to write accurate SQL queries based on the given database schema and user question. Adhere to these guidelines:

Input Format:

Schema: A precise representation of the database schema in SQL format.
Question: A natural language question to be answered using the schema.
Output Requirements:

SQL Query Only: Return only the SQL query without any additional text or explanation.
Query Style: Ensure queries are optimized for readability and efficiency, using table aliases where appropriate.

Example Input-Output Pairs:
{{
    Input:
    {{
        Schema: CREATE TABLE employees (id INT, name VARCHAR, age INT, department_id INT); CREATE TABLE departments (id INT, name VARCHAR);
        Question: "Find the names of employees in the 'HR' department."
    }}

    Output:
    {{
        SELECT e.name FROM employees AS e JOIN departments AS d ON e.department_id = d.id WHERE d.name = 'HR';
    }}
}}

{{
    Input:
    {{
        Schema: CREATE TABLE sales (id INT, product_name VARCHAR, amount DECIMAL, date DATE);
        Question: "Calculate the total sales amount for 'Product A' in 2023."
    }}

    Output:
    {{
        SELECT SUM(amount) FROM sales WHERE product_name = 'Product A' AND YEAR(date) = 2023;
    }}
}}

{{
    Input:
    {{
        Schema: CREATE TABLE users (id INT, city VARCHAR); CREATE TABLE orders (id INT, user_id INT, total DECIMAL);
        Question: "Find the total order value of users from 'New York'."
    }}

    Output:
    {{
        SELECT SUM(o.total) FROM orders AS o JOIN users AS u ON o.user_id = u.id WHERE u.city = 'New York';
    }}
}}

### Input:
Question : {input}
Database Schema : {context}

### Response:
{output}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):

    inputs       = examples["question"]
    contexts      = examples["context"]
    outputs      = examples["answer"]
    texts = []
    for context, input, output in zip(contexts, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input=input, context=context, output=output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


from datasets import load_dataset
dataset = load_dataset("b-mc2/sql-create-context", split = "train")


dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [21]:
dataset

Dataset({
    features: ['answer', 'question', 'context', 'text'],
    num_rows: 78577
})

In [15]:
print(dataset[0]['text'])



### Instruction:
You are an SQL Expert. Your task is to write accurate SQL queries based on the given database schema and user question. Adhere to these guidelines:

Input Format:

Schema: A precise representation of the database schema in SQL format.
Question: A natural language question to be answered using the schema.
Output Requirements:

SQL Query Only: Return only the SQL query without any additional text or explanation.
Query Style: Ensure queries are optimized for readability and efficiency, using table aliases where appropriate.

Example Input-Output Pairs:
{
    Input: 
    {
        Schema: CREATE TABLE employees (id INT, name VARCHAR, age INT, department_id INT); CREATE TABLE departments (id INT, name VARCHAR);
        Question: "Find the names of employees in the 'HR' department."
    }

    Output:
    {
        SELECT e.name FROM employees AS e JOIN departments AS d ON e.department_id = d.id WHERE d.name = 'HR';
    }
}

{
    Input:
    {
        Schema: CREATE TABLE 

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [16]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 5,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # num_train_epochs = 2,
        max_steps=100,
        learning_rate = 2e-3,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/78577 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.971 GB of memory reserved.


In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 78,577 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 5 | Gradient Accumulation steps = 2
\        /    Total batch size = 10 | Total steps = 100
 "-____-"     Number of trainable parameters = 60,555,264


Step,Training Loss
1,1.114600
2,1.111300
3,0.999100
4,0.759700
5,0.360700
6,0.315500
7,0.255600
8,0.209400
9,0.209700
10,0.170900


Step,Training Loss
1,1.114600
2,1.111300
3,0.999100
4,0.759700
5,0.360700
6,0.315500
7,0.255600
8,0.209400
9,0.209700
10,0.170900


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

500.2925 seconds used for training.
8.34 minutes used for training.
Peak reserved memory = 3.342 GB.
Peak reserved memory for training = 1.057 GB.
Peak reserved memory % of max memory = 22.661 %.
Peak reserved memory for training % of max memory = 7.167 %.


<a name="Inference"></a>
### Inference
Let's run the model! Since we're using `Phi-3`, use `apply_chat_template` with `add_generation_prompt` set to `True` for inference.

In [20]:
qs = "List the states where both the secretary of 'Treasury' department and the secretary of 'Homeland Security' were born."
table_schema = "CREATE TABLE management (department_id VARCHAR, head_id VARCHAR); CREATE TABLE head (born_state VARCHAR, head_id VARCHAR); CREATE TABLE department (department_id VARCHAR, name VARCHAR)"

# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        input=qs, # instruction
        context=table_schema, # input
        output="", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['\n\n### Instruction:\nYou are an SQL Expert. Your task is to write accurate SQL queries based on the given database schema and user question. Adhere to these guidelines:\n\nInput Format:\n\nSchema: A precise representation of the database schema in SQL format.\nQuestion: A natural language question to be answered using the schema.\nOutput Requirements:\n\nSQL Query Only: Return only the SQL query without any additional text or explanation.\nQuery Style: Ensure queries are optimized for readability and efficiency, using table aliases where appropriate.\n\nExample Input-Output Pairs:\n{\n    Input: \n    {\n        Schema: CREATE TABLE employees (id INT, name VARCHAR, age INT, department_id INT); CREATE TABLE departments (id INT, name VARCHAR);\n        Question: "Find the names of employees in the \'HR\' department."\n    }\n\n    Output:\n    {\n        SELECT e.name FROM employees AS e JOIN departments AS d ON e.department_id = d.id WHERE d.name = \'HR\';\n    }\n}\n\n{\n    Input:\

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [34]:
from google.colab import userdata

In [35]:
# model.save_pretrained("lora_model") # Local saving
# tokenizer.save_pretrained("lora_model")
model.push_to_hub("Vishal-Sagar/Qwen2.5_7B-text-to-sql-basic", token = userdata.get('HF_TOKEN')) # Online saving
tokenizer.push_to_hub("Vishal-Sagar/Qwen2.5_7B-text-to-sql-basic", token = userdata.get('HF_TOKEN')) # Online saving

README.md:   0%|          | 0.00/579 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Vishal-Sagar/Qwen2.5_7B-text-to-sql-basic


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
# # prompt: make zip folder of lora_model and download it

# !zip -r lora_model.zip lora_model
# !ls
# from google.colab import files
# files.download("lora_model.zip")

  adding: lora_model/ (stored 0%)
  adding: lora_model/adapter_config.json (deflated 56%)
  adding: lora_model/tokenizer.json (deflated 85%)
  adding: lora_model/README.md (deflated 66%)
  adding: lora_model/tokenizer_config.json (deflated 84%)
  adding: lora_model/tokenizer.model (deflated 55%)
  adding: lora_model/special_tokens_map.json (deflated 76%)
  adding: lora_model/added_tokens.json (deflated 62%)
  adding: lora_model/adapter_model.safetensors (deflated 8%)
huggingface_tokenizers_cache  lora_model  lora_model.zip  outputs  sample_data


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>